In [ ]:
!pip3 install karateclub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 321 kB/s 
     |████████████████████████████████| 1.8 MB 28.9 MB/s 
     |████████████████████████████████| 24.1 MB 2.9 MB/s 
     |████████████████████████████████| 50 kB 4.3 MB/s 
  Created wheel for karateclub: filename=karateclub-1.3.0-py3-none-any.whl size=98587 sha256=e0e05df75b73df036b55d7b19d6cb49cad90e912b9752e9deef74f72604f70d1
  Stored in directory: /root/.cache/pip/wheels/5c/e7/85/fb61ed4735fef344e737483ac55d4900bbc9323fd500c25183
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149875 sha256=fa3cd557194ed40449cb986a89357b3ff85240bc0fa3642ccc368a9832549bd1
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built karateclub python-Levenshtein
  Attempting uninstall: gensim
    Found existing installation: g

In [ ]:
!pip install stellargraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 435 kB 34.4 MB/s 


In [ ]:
import os
import csv
import pdb
import math
import time 
import pickle
import pickle
import random
import warnings
import datetime 
import collections
import numpy as np 
import pandas as pd  
import xgboost as xgb
import networkx as nx
from tqdm.notebook import tqdm
import matplotlib.pylab as plt 
from matplotlib import rcParams   
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import time
import collections

# Algo import
from stellargraph.mapper import GraphWaveGenerator
from stellargraph import StellarGraph
from scipy.sparse.linalg import eigs

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_path = 'gdrive/My Drive/Major Project/'

In [ ]:
combined_tr = pd.read_csv(data_path+'/train_91.csv', index_col = [0])
print(combined_tr.shape)
combined_tr.head()

(902474, 3)


,src,dest,link
73132,35080,10701,0
404654,54809,75095,0
310475,67628,60404,0
301271,57253,462,0
332230,15167,47459,1


In [ ]:
combined_te = pd.read_csv(data_path+'/test_91.csv', index_col = [0])
print(combined_te.shape)
combined_te.head()

(106363, 3)


,src,dest,link
487942,32198,62377,0
103900,6056,11650,1
78661,2040,22467,1
71074,143,0,1
382438,1609,5035,1


In [ ]:
df_train = pd.read_csv(data_path+'/pos_train_91.csv', index_col = [0])
print(df_train.shape)
df_train.head()

(452474, 3)


,src,dest,link
496205,54500,16485,1
141147,3318,30547,1
483066,33514,4007,1
478351,28118,2641,1
505144,69985,61687,1


In [ ]:
train_graph = nx.from_pandas_edgelist(df_train, source='src', target='dest', create_using=nx.MultiDiGraph)

In [ ]:
scales = [5, 10, 20, 50]
sample_points = np.linspace(0, 100, 50).astype(np.float32)
degree = 20
G = StellarGraph.from_networkx(train_graph)
generator = GraphWaveGenerator(G, scales=scales, degree=degree)

In [ ]:
emb = generator.flow(
    node_ids = G.nodes(), sample_points = sample_points, batch_size = 1, repeat = False
)

In [ ]:
embeddings = [x.numpy() for x in tqdm(emb)]

  0%|          | 0/75879 [00:00<?, ?it/s]

In [ ]:
filename = data_path + 'epinions_graphwave_embeddings_91'
outfile = open(filename, "wb")
pickle.dump(embeddings, outfile)

In [ ]:
filep = open(filename, "rb")
gw_emb = pickle.load(filep)
filep.close()

In [ ]:
emb_dict = {}
for node, emb in zip(G.nodes(), gw_emb):
  emb_dict[node] = emb
print(len(emb_dict))

75879


In [ ]:
# get training embeddings
train_emb = []
for i, row in tqdm(combined_tr.iterrows()):
  comb_emb = emb_dict[row['src']][0] * emb_dict[row['dest']][0]
  train_emb.append(comb_emb)
train_emb = np.array(train_emb)
print(train_emb.shape)
train_y = combined_tr['link']
print(len(train_y))

0it [00:00, ?it/s]

(902474, 400)
902474


In [ ]:
# get test embeddings
test_emb = []
for i, row in tqdm(combined_te.iterrows()):
  comb_emb = emb_dict[row['src']][0] * emb_dict[row['dest']][0]
  test_emb.append(comb_emb)

test_emb = np.array(test_emb)
print(test_emb.shape)
test_y = combined_te['link'] 
print(len(test_y))

0it [00:00, ?it/s]

(106363, 400)
106363


In [ ]:
pickle.dump((train_emb, train_y), open(data_path + 'epinions_graphwave_train91.pkl', 'wb'))
pickle.dump((test_emb, test_y), open(data_path + 'epinions_graphwave_test91.pkl', 'wb'))

In [ ]:
train_emb, train_y = pickle.load(open(data_path + 'epinions_graphwave_train91.pkl', 'rb'))
test_emb, test_y = pickle.load(open(data_path + 'epinions_graphwave_test91.pkl', 'rb'))

In [ ]:
train_emb_sample, _, train_y_sample, _ = train_test_split(train_emb, train_y, stratify = train_y, test_size = 0.5, random_state = 42)

In [ ]:
clf = XGBClassifier(max_depth=7, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb_sample, train_y_sample)
print("Time taken for XGB fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y_sample,clf.predict(train_emb_sample))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for XGB fit =  945.2995896339417
Time taken for train data precition =  3.0100271701812744
Time taken for test data precition =  0.8517124652862549
Train Score 0.7741833489918336 test Score 0.792232126614517


In [ ]:
clf = RandomForestClassifier(max_depth=12, n_estimators=100, n_jobs=-1,
              random_state=25)
start = time.time()
clf.fit(train_emb_sample, train_y_sample)
print("Time taken for RandomForest fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y_sample,clf.predict(train_emb_sample))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for RandomForest fit =  524.3134183883667
Time taken for train data precition =  1.4322285652160645
Time taken for test data precition =  0.37732458114624023
Train Score 0.6678839335531697 test Score 0.6927350269778646


In [ ]:
clf = LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, 
                     objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                     subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=- 1, importance_type='split')
start = time.time()
clf.fit(train_emb_sample, train_y_sample)
print("Time taken for LGBM fit = ", time.time() - start)
start = time.time()
train_sc = f1_score(train_y_sample,clf.predict(train_emb_sample))
print("Time taken for train data precition = ", time.time() - start)
start = time.time()
test_sc = f1_score(test_y,clf.predict(test_emb))
print("Time taken for test data precition = ", time.time() - start)
print('Train Score',train_sc,'test Score',test_sc)

Time taken for LGBM fit =  25.46351981163025
Time taken for train data precition =  2.417412519454956
Time taken for test data precition =  0.5640771389007568
Train Score 0.7704404795346014 test Score 0.7904564882574343
